In [1]:
import dustpy
from dustpy import Simulation
from dustpy import constants as c
from dustpy import plot
from dustpy import hdf5writer

from setup_externalPhotoevaporation import setup_externalPhotoevaporation_FRIED
from setup_externalPhotoevaporation import setup_lostdust
from setup_externalPhotoevaporation import setup_gasonly

import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import astropy
from astropy.table import Table
from matplotlib.colors import ListedColormap

In [2]:
def ini_sim(mstar, mdisc, rc, fuv):   #mstar [Msun], mdisc [Msun], rc [AU], fuv [G0]
    sim = Simulation()
    sim.ini.star.M = mstar*c.M_sun
    
    sim.ini.gas.Mdisk = mdisc*c.M_sun        
    sim.ini.gas.SigmaRc = rc*c.au          
    sim.ini.gas.gamma = 1.0  
    sim.ini.gas.alpha = 1e-3
    
    # Relevant Dust parameters
    sim.ini.dust.d2gRatio = 0.01                # Initial dust-to-gas ratio              
    sim.ini.dust.vfrag = 1000.0                 # Dust fragmentation velocity [cm/s]

    sim.ini.grid.Nr = 500
    sim.ini.grid.rmin = 5*c.au
    sim.ini.grid.rmax = 1000*c.au
    sim.ini.grid.Nmbpd = 7
    sim.ini.grid.mmin = 1.e-12
    sim.ini.grid.mmax = 1.e+5

    option_sqrt_grid = True         
    option_track_lostdust = False
    option_gasonly = True

    if option_gasonly:
        sim.ini.grid.Nmbpd = 2
        sim.ini.grid.mmax = 1e-10
    
    if option_sqrt_grid:
        sim.grid.ri = np.square(np.linspace(np.sqrt(sim.ini.grid.rmin), np.sqrt(sim.ini.grid.rmax), num = sim.ini.grid.Nr +1))

    sim.initialize()
    setup_externalPhotoevaporation_FRIED(sim, fried_filenames = ["./FRIEDV2_0p1Msol_fPAH1p0_growth.dat","./FRIEDV2_0p3Msol_fPAH1p0_growth.dat","./FRIEDV2_0p6Msol_fPAH1p0_growth.dat","./FRIEDV2_1p0Msol_fPAH1p0_growth.dat","./FRIEDV2_1p5Msol_fPAH1p0_growth.dat","./FRIEDV2_3p0Msol_fPAH1p0_growth.dat"], UV_Flux = fuv)
           
    T_sun = 5772.
    L_sun = 4. * c.pi * c.R_sun**2 * c.sigma_sb * T_sun**4
            
    sim.star.L = 0.3 * L_sun       # as a representative value for our stars
    sim.star.L.updater = None
    
    if option_track_lostdust:
        setup_lostdust(sim, using_FRIED = True)

    if option_gasonly:
        setup_gasonly(sim)

    return sim

In [3]:
snapshots1 = np.logspace(3,6,9,endpoint=False)
snapshots2 = np.linspace(1, 10, 19) * 1e+6

sim = ini_sim(0.6, 0.06, 10, 100)  #mstar mdisc rc fuv
sim.t.snapshots = np.concatenate((snapshots1, snapshots2))* c.year

sim.writer.datadir='test_dir'
sim.writer.dumping = False
sim.writer.overwrite = True
sim.verbosity = 2
    
sim.run()


DustPy v1.0.5

Documentation: https://stammler.github.io/dustpy/
PyPI:          https://pypi.org/project/dustpy/
GitHub:        https://github.com/stammler/dustpy/

Please cite Stammler & Birnstiel (2022).

Checking for mass conservation...

    - Sticking:
        max. rel. error:  5.38e-01
        for particle collision
            m[1] =  3.16e-12 g    with
            m[1] =  3.16e-12 g
    - Full fragmentation:
        max. rel. error:  2.22e-16
        for particle collision
            m[1] =  3.16e-12 g    with
            m[2] =  1.00e-11 g
    - Erosion:
        max. rel. error:  2.22e-16
        for particle collision
            m[0] =  1.00e-12 g    with
            m[3] =  3.16e-11 g

Creating data directory 'test_dir'.
Writing file test_dir/data0000.hdf5
Writing file test_dir/data0001.hdf5
Writing file test_dir/data0002.hdf5
Writing file test_dir/data0003.hdf5
Writing file test_dir/data0004.hdf5
Writing file test_dir/data0005.hdf5
Writing file test_dir/data0006.hdf5
Wri

In [5]:
reader=hdf5writer()
reader.datadir = 'test_dir'  #sim.writer.datadir
time = reader.read.sequence('t')
r = reader.read.sequence('grid.r')
Sigma_g = reader.read.sequence('gas.Sigma')
Sigma_dot = reader.read.sequence('gas.S.ext')
ring_area = reader.read.sequence('grid.A')
mass_gas = (ring_area * Sigma_g).sum(axis = -1) 
mass_loss = -(ring_area * Sigma_dot).sum(axis = -1) 
rtrunc = reader.read.sequence('FRIED.rTrunc')
vrad = reader.read.sequence('gas.v.rad')
accretion_rate = -2*np.pi*Sigma_g * vrad * r / (c.M_sun/c.year)

cum_mass_gas=[]
for i in range(0,len(time)):
    cum_mass_gas.append( np.cumsum(ring_area[i] * Sigma_g[i]) )
